# Métodos de ensamblado


![title](img/intro.png)

Introducción del artículo *González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

IMAGEN PEZ

Los métodos de ensamblado entrenan múltiples clasificadores (weak learners) para el mismo problema y combinan los resultados (formando un strong learner). El esquema básico sería:

![architecture](img/ensemble_architecture.png)

Extraída del libro *Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

Un ensamblado está formado por distintos clasificadores como puede ser un árbol de decisión, una regresión logística u otros tipos de modelos (tanto para el caso de clasificación como el de regresión).

Un ensamblado se construye en dos pasos:
 - generando los modelos base (*weak learners*)
 - y combinándolos después.

¿Coste computacional?
Para seleccionar un único modelo hay que probar varios y elegir bien los parámetros. Esto sería parecido a entrenar distintos modelos base. La combinación final es simple y, por tanto, tiene bajo coste computacional.

¿Cómo combinar los resultados de los modelos?
Media, media ponderada, voto de la mayoría, etc.

## Tipos de ensamblado

Existen distintas taxonomías de ensamblado de modelos pero los tres tipos más conocidos y usados son:
- Bagging
- Boosting
- Stacking

### Stacking

Consiste en combinar distintos modelos a través de otro modelo (un meta-modelo). Este útimo modelo usa como variables los outputs de los modelos iniciales para obtener una predicción final.


![stacking_scheme](img/stacking.png)

Imagen obtenida de [Jason Brownlee - machinelearningmastery](https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/).

En Python: [StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) y [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html).

### Bagging (Bootstrao Aggregation)

Es un método de ensamblado que entrena en paralelo el mismo tipo de modelo. Una de las motivaciones principales de los modelos de ensamblado en paralelo es aprovechar la independencia de los clasificadores para mejorar el error. Sin embargo, al basarse todos los modelos en los mismos datos, se pierde algo de esa independencia. Por ello, se introduce aleatoriedad en los datos mediante el muestreo con *bootstrap*.

Bootstrap consiste en obtener muestras con reemplazamiento del mismo tamaño que la muestra original.

Bagging consiste en entrenar el mismo modelo en distintas muestras bootstrap del mismo conjunto de datos y luego promediar las predicciones de todos ellos puesto que se reduce la varianza.

La predicción final se puede obtener como un promedio, por votación, etc.

*Out-of-bag error*. Como no todas las muestras se usan para entrenar el modelo, se puede estimar el error con dichas muestras.

![bagging_inffus](img/bagging_inffus.png)

*González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

### Boosting

Es un método de ensamblado que entrena de forma secuencial el mismo tipo de modelo. Cada modelo tiene en cuenta los errores de los modelos previos a través de pesos que le dan más importancia a aquellas observaciones que se han sido clasificadas de forma incorrecta anteriormente.

Procedimiento general del método Boosting:

![boosting_procedure](img/boosting_procedure.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

![boosting_inffus](img/boosting_inffus.png)

*González, S., García, S., Del Ser, J., Rokach, L., & Herrera, F. (2020). A practical tutorial on bagging and boosting based ensembles for machine learning: Algorithms, software tools, performance study, practical perspectives and opportunities. Information Fusion, 64, 205-237.*

El algoritmo anterior muestra el procedimiento general de Boosting, para cada algoritmo en concreto hay que definir *Adjust Distribution* y *Combine Outputs*. Dos de los principales algoritmos son:
- AdaBoost.
- Gradient Boosting.

Ejemplos sencillos de AdaBoost:

![xor_ada](img/adaboost_xor.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

![gaussian_ada](img/gaussian_adaboost.png)

*Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.*

## Ejemplo de Boosting en Python

- Gradient Boosting Machine (GBM)
- Extreme Gradient Boosting
- CatBoost. Acepta variables categóricas. [Tutorial en Python](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb)
- HistGradientBoostingClassifier. Más eficiente. También acepta variables categóricas.


In [29]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [40]:
pima = pd.read_csv('pima.csv')
pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,class
0,14,175,62,30,0,33.6,0.212,38,1
1,4,146,78,0,0,38.5,0.520,67,1
2,15,136,70,32,110,37.1,0.153,43,1
3,3,107,62,13,48,22.9,0.678,23,1
4,3,169,74,19,125,29.9,0.268,31,1


In [31]:
y = pima['class'].to_numpy()
X = pima.iloc[:,0:-1].to_numpy() # de 0 hasta la penúltima porque es la y

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42, stratify = y)

In [33]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7402597402597403

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7575757575757576

In [36]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01, use_label_encoder = False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.7792207792207793

In [46]:
# categorizamos edad
bins = [0, 30, 40, 50, 60, 70, 150]
group_names = [1,2,3,4,5,6]
pima['age_cat'] = pd.cut(pima[' Age'], bins, labels=group_names)

In [138]:
pima2 = pima.drop(columns=" Age")

In [139]:
y = pima2['class']
X = pima2.drop(columns="class")

In [140]:
X

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,age_cat
0,14,175,62,30,0,33.6,0.212,2
1,4,146,78,0,0,38.5,0.520,5
2,15,136,70,32,110,37.1,0.153,3
3,3,107,62,13,48,22.9,0.678,1
4,3,169,74,19,125,29.9,0.268,2
...,...,...,...,...,...,...,...,...
763,5,117,92,0,0,34.1,0.337,2
764,4,83,86,19,0,29.3,0.317,2
765,7,119,0,0,0,25.2,0.209,2
766,1,95,66,13,38,19.6,0.334,1


In [141]:
categorical_features_indices = np.where(X.dtypes == 'category')[0]
print(categorical_features_indices)

[7]


In [143]:
print(X.dtypes)



Preg          int64
 Plas         int64
 Pres         int64
 Skin         int64
 Insu         int64
 Mass       float64
 Pedi       float64
age_cat    category
dtype: object


In [131]:
print(X)

None


In [144]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42, stratify = y)

In [146]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()
model.fit(X_train,y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test))
y_pred = model.predict(X_test)
# metrics.accuracy_score(y_pred,y_test)

Learning rate set to 0.027178
0:	learn: 0.6808677	test: 0.6822882	best: 0.6822882 (0)	total: 1.38ms	remaining: 1.38s
1:	learn: 0.6690213	test: 0.6722366	best: 0.6722366 (1)	total: 2.5ms	remaining: 1.25s
2:	learn: 0.6607610	test: 0.6643337	best: 0.6643337 (2)	total: 3.56ms	remaining: 1.18s
3:	learn: 0.6488270	test: 0.6526545	best: 0.6526545 (3)	total: 4.54ms	remaining: 1.13s
4:	learn: 0.6372987	test: 0.6411922	best: 0.6411922 (4)	total: 5.41ms	remaining: 1.08s
5:	learn: 0.6274098	test: 0.6330198	best: 0.6330198 (5)	total: 6.34ms	remaining: 1.05s
6:	learn: 0.6180888	test: 0.6234953	best: 0.6234953 (6)	total: 7.28ms	remaining: 1.03s
7:	learn: 0.6092084	test: 0.6161624	best: 0.6161624 (7)	total: 8.13ms	remaining: 1.01s
8:	learn: 0.6000055	test: 0.6086445	best: 0.6086445 (8)	total: 9.02ms	remaining: 993ms
9:	learn: 0.5901614	test: 0.6012433	best: 0.6012433 (9)	total: 9.88ms	remaining: 978ms
10:	learn: 0.5815915	test: 0.5936135	best: 0.5936135 (10)	total: 10.8ms	remaining: 967ms
11:	learn: 0

In [149]:
y_pred = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(y_pred[:10])
print(predictions_probs[:10])

[0 0 0 1 0 0 0 0 0 0]
[[0.53699548 0.46300452]
 [0.58453892 0.41546108]
 [0.60352569 0.39647431]
 [0.27513762 0.72486238]
 [0.63438393 0.36561607]
 [0.87096668 0.12903332]
 [0.80605339 0.19394661]
 [0.63598754 0.36401246]
 [0.50412505 0.49587495]
 [0.55378988 0.44621012]]


In [148]:
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

In [150]:
compute_accuracy(y_test, y_pred)

0.7705627705627706

In [156]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [157]:
mod = HistGradientBoostingClassifier(max_bins=255, max_iter=100, categorical_features = categorical_features_indices)


In [159]:
mod.fit(X_train, y_train)
y_pred = mod.predict(X_test)


In [161]:
compute_accuracy(y_test, y_pred)

0.7445887445887446